In [112]:
#Import Libraries
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, interactive, fixed, FloatSlider
import ipywidgets as widgets

%run conservation_equations.ipynb
#%run Interactive.ipynb

In [113]:
def pressure_drop(z1, z2, Diameter, MassFlow, InletPressure, Length, Elbow_90, GateValve_FullyOpen, \
                  ThermalConductivity, InsulationThickness):
    '''Calculate parameters from steam table'''
    sat_steam = IAPWS97(P = InletPressure, x=1)
    temperature = sat_steam.T
    density = sat_steam.rho
    enthalpy =  sat_steam.h
    dynamic_viscosity = sat_steam.mu
    #print(temperature, density, dynamic_viscosity, enthalpy)
    
    '''Calculate Derived Parameters'''
    Diameter = Diameter * 0.0254
    #Relative roughness
    roughness = 0.000046
    relative_roughness = roughness / Diameter
    #cross sectional area
    area = math.pi * Diameter**2 / 4
    #for straight pipe
    area2 = area
    #velocity
    velocity = MassFlow / (density * area)
    #print(velocity, area)
    
    '''Skin Friction Calculation'''
    nRE = reynolds_number(Diameter, velocity, density, dynamic_viscosity)
    friction_factor = ff_colebrook(nRE, relative_roughness)
    friction_skin = f_skin(temperature, Diameter, velocity, relative_roughness, Length)
    
    '''Pipe Enlargement and contraction '''
    friction_expansion = f_expansion(area, area2, velocity)
    friction_contraction = f_contraction(area, area2, velocity)
    
    """Define Fittings"""
    #Represent in a list of 8
    fittings_list = [0, 0, 0, 0, 0, 0, 0, 0]
    fittings_list[1] = Elbow_90
    fittings_list[6] = GateValve_FullyOpen
    #fittings [0] = elbow_45  #fittings [1] = elbow_90
    #fittings [2] = tee       #fittings [3] = return_bend
    #fittings [4] = coupling  #fittings [5] = union
    #fittings [6] = gate_valve_full  #fittings [7] = gate_valve_half
    '''Losses in fittings and valves'''
    friction_fittings = f_fittings(velocity, fittings_list)
    #print (velocity, friction_fittings)
    
    '''Calulate total friction losses'''
    friction_total = friction_skin + friction_expansion + friction_contraction + friction_fittings
    
    
    '''Define Parameters for heat loss'''
    #Convective heat transfer coefficient (h; W/m**K)
    h_steam_pipe = 36
    h_insulation_air = 7
    #Insulation
    t_insulation = InsulationThickness * 0.0254    #insulation thickness (m)
    k_insulation = ThermalConductivity          #insulation's thermal conductivity (W/m** K)
    #Pipe
    t_pipe = 0.5 * 0.0254         #pipeline thickness (m)
    k_pipe = 56           #pipe's thermal conductivity (W/m** K)
    #id_pipe = 0.25        #pipe's inner diameter (m) given at diameter
    #l_pipe = 6000         #length of pipeline (m) given at length
    #Temp
    t1 = temperature - 273.15           #temp inside the pipeline
    t2 = 25                             #temp; atmospheric
    
    r1 = Diameter
    r2 = Diameter + t_pipe
    r3 = Diameter + t_pipe + t_insulation
    
    h_t = heat_transfer(Length, t1, t2, h_steam_pipe, h_insulation_air, k_pipe, k_insulation, r1, r2, r3)
    Q = h_t / MassFlow
    
    
    '''Calculate gdZ'''
    gdz = head(z1, z2)

    
    '''Calculate outlet pressure'''
    udu = 0
    p2 = p_out(InletPressure, friction_total, gdz, udu, density, Q )
    t_loss = friction_total - Q + gdz
    
    print('{:26}'.format('Velocity (m/s)'), '{:>16.2f}'.format(velocity))
    print('{:26}'.format('Tot Loss (J/kg)'), '{:>16.2f}'.format(t_loss))
    print('{:26}'.format('Heat Loss (J/kg)'), '{:>16.2f}'.format(-Q))
    print ('{:26}'.format('Head (J/kg)'), '{:>16.2f}'.format(gdz))
    print ('{:26}'.format('Friction Loss (J/kg)'), '{:>16.2f}'.format(friction_total))
    print ('{:26}'.format('  Skin Friction (J/kg)'), '{:>16.2f}'.format(friction_skin))
    print ('{:26}'.format('  Fittings Friction (J/kg)'), '{:>16.2f}'.format(friction_fittings))
    print ('{:26}'.format('Outlet Pressure (Mpa)'), '{:>16.2f}'.format(p2))

In [114]:
z1_widget = FloatSlider(min=500.0, max=700.0, step=5, value = 655)
z2_widget = FloatSlider(min=500.0, max=700.0, step=5, value = 613)
dia_widget = FloatSlider(min=1.0, max=42.0, step=1, value = 29)#.24961)
mf_widget = FloatSlider(min=1.0, max=100.0, step=1, value = 51)
inpressure_widget = FloatSlider(min=0.60, max=1.0, step=0.01, value=0.8862)
length_widget = FloatSlider(min=10, max=5000, step=10, value=580)
elbow90_widget = FloatSlider(min=0, max=80, step=1, value=8)
gv_open_widget = FloatSlider(min=0, max=10, step=1, value=1)
x_widget = FloatSlider(min=0.01, max=0.2, step=0.005, value = 0.05)      #thermal conductivity 0 to 0.1
y_widget = FloatSlider(min=0.0, max=8.0, step=0.1, value=2.0)            #thickness 0 to 10

interact(pressure_drop, z1= z1_widget, z2= z2_widget, Diameter= dia_widget, MassFlow = mf_widget, \
        InletPressure = inpressure_widget,\
        Length= length_widget, Elbow_90=elbow90_widget, GateValve_FullyOpen = gv_open_widget, \
        ThermalConductivity=x_widget, InsulationThickness=y_widget)

Velocity (m/s)                        26.10
Tot Loss (J/kg)                    10679.24
Heat Loss (J/kg)                    6998.38
Head (J/kg)                          412.02
Friction Loss (J/kg)                3268.84
  Skin Friction (J/kg)              3268.84
  Fittings Friction (J/kg)             0.00
Outlet Pressure (Mpa)                  0.84


In [115]:
class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = {}

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]

class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        #self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

class Node:
    def __init__(self, pressure, mass_flow):
        self.press = pressure
        self.elev = elevation
        self.mf = mass_flow
        
class PipeLine:
    def __init__(self, length, diameter):
        self.length = length
        self.dia = diameter
        
g=Graph()
g.add_vertex('504')
g.add_vertex('505')
g.add_vertex('701')
g.add_vertex('IF')

g.add_edge('504', '701', 89)
g.add_edge('505', '701', 69)
g.add_edge('701', 'IF', 39)
g.add_edge('504', '702', 139)


print (g.vert_dict.keys())
print (g.get_vertices())

for v in g:
    for w in v.get_connections():
        vid = v.get_id()
        wid = w.get_id()
        print ('( %s , %s, %3d)'  % ( vid, wid, v.get_weight(w)))

for v in g:
    print ('g.vert_dict[%s]=%s' %(v.get_id(), g.vert_dict[v.get_id()]))
    #str(g.vert_dict['504'].id) + ' adjacent: ' + str([x.id for x in g.vert_dict['701'].adjacent])
    #print(v.adjacent.keys)
print (g.vert_dict[v.get_id()])
print ('amf', g.vert_dict['701'])
str(g.vert_dict['504'].id) + ' adjacent: ' + str([x.id for x in g.vert_dict['701'].adjacent])

#print('full dict:', g.vert_dict)
print ('vertex :', g.vert_dict['504'].id)   #direct access the function get_id of Vertex Class
print ('vertex :', g.vert_dict['504'])     #access __str__ function of Vertex Class


dict_keys(['504', '701', 'IF', '702', '505'])
dict_keys(['504', '701', 'IF', '702', '505'])
( 504 , 702, 139)
( 504 , 701,  89)
( 701 , IF,  39)
( 505 , 701,  69)
g.vert_dict[504]=504 adjacent: ['702', '701']
g.vert_dict[701]=701 adjacent: ['IF']
g.vert_dict[IF]=IF adjacent: []
g.vert_dict[702]=702 adjacent: []
g.vert_dict[505]=505 adjacent: ['701']
505 adjacent: ['701']
amf 701 adjacent: ['IF']
vertex : 504
vertex : 504 adjacent: ['702', '701']
